# Agent multi tool space setup 

Steps to execute

1. Load local files into UC tables
2. Create genie space and hydrate with datasets 
3. Create vector store
4. Set mlflow experiment
5. Write results into app_config.yaml file. This will be used by the streamlit app


In [ ]:
%pip install --quiet -U databricks-sdk==0.49.0 "databricks-langchain>=0.4.0" databricks-agents mlflow[databricks] langchain==0.3.25 langchain_core==0.3.59 databricks-vectorsearch==0.55 pydantic==2.10.1 unitycatalog-ai==0.3.0 unitycatalog-client
dbutils.library.restartPython()

### Set global variables

Make these your own values


In [ ]:
catalog = "main"
schema = "genie_multi_agent"
warehouse_cluster_id = "072b588d901e6eed"

databricks_foundation_model = "databricks-claude-3-7-sonnet"

vector_search_endpoint_name = "vector-search-multi-agent-genie"


### Load local files into UC tables

In [0]:
import os
from pathlib import Path

import pandas as pd


file_names = [
    i
    for i in os.listdir("data")
    if i.endswith(".csv")
    if i != "retail_code_of_conduct.csv"
]
table_names = [i.replace(".csv", "") for i in file_names]

spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}")

existing_tables = [
    i.tableName
    for i in list(
        spark.sql(f"SHOW TABLES FROM {catalog}.{schema}").select("tableName").collect()
    )
]

for file_name, table_name in zip(file_names, table_names):
    if table_name not in existing_tables:
        full_filepath = f"file://{(Path('.') / 'data' / file_name).absolute()}"
        full_table_path = f"{catalog}.{schema}.{table_name}"
        df = spark.read.csv(full_filepath, header=True, inferSchema=True)
        df.write.mode("overwrite").saveAsTable(full_table_path)

if "retail_code_of_conduct" not in existing_tables:
    code_of_conduct = pd.read_csv("data/retail_code_of_conduct.csv")

    code_of_conduct_spark = spark.createDataFrame(code_of_conduct)
    code_of_conduct_spark.write.mode("overwrite").saveAsTable(
        f"{catalog}.{schema}.retail_code_of_conduct"
    )

No such comm: LSP_COMM_ID
No such comm: LSP_COMM_ID


### Create Genie Space

Sneaky sneaky, genie spaces aren't supported but that doesn't mean we can't just look up some api calls via developer tools ;) 

https://databricks.aha.io/ideas/ideas/DB-I-10709?active_tab=idea_comments

In [0]:
payload_products_inventory_genie_room = {
    "display_name": "retail-product-inventory-snapshot",
    "description": "This genie room provides information about products and the current inventory snapshot across stores.",
    "table_identifiers": [
        f"{catalog}.{schema}.retail_inventory_snapshot",
        f"{catalog}.{schema}.retail_products",
    ],
    "run_as_type": "VIEWER",
    "warehouse_id": warehouse_cluster_id,
}

payload_sales_store_genie_room = {
    "display_name": "retail-sales-store-performance",
    "description": "This genie space allows users to ask questions about store performance and inventory positions. This could be forecasts, sales, buy-online-pick-up-in-store etc.",
    "table_identifiers": [
        f"{catalog}.{schema}.retail_sales_monthly",
        f"{catalog}.{schema}.retail_stores",
    ],
    "run_as_type": "VIEWER",
    "warehouse_id": warehouse_cluster_id,
}

payload_sales_text_instruction = {
    "title": "Notes",
    "content": "When forecasts are requested, you must use the sql query provided to generate forecasts.",
    "instruction_type": "TEXT_INSTRUCTION",
}


payload_sales_sql_query = {
    "title": "Example query to generate sales forecasts by store for the next N. the following is for store 110 forecasts for the next 6 months.",
    "content": f"with store_data as (\nSELECT * FROM {catalog}.{schema}.retail_sales_monthly where store_id = 110 order by month --replace the store id here for other stores\n)\nSELECT * FROM ai_forecast(\n  TABLE(store_data),\n  horizon => (select max(month) + interval 7 month from store_data), --the interval can be any positive int. its always requested horizon+1\n  time_col => 'month', -- what is the time dim\n  value_col => 'total_sales' -- the metric we need to forecast\n);",
    "instruction_type": "SQL_INSTRUCTION",
    "parameters": [],
    "usage_guidance": "When a forecast is requested",
}


In [0]:
from typing import Any
import time

from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
route = "/api/2.0/data-rooms/"
data_rooms = w.api_client.do(method="GET", path=route)["data_rooms"]


def _post_sales_forecast_instructions(
    genie_space_id: str,
    payload_text_instructions: dict[str, str],
    pyaload_sql_instructions: dict[str, Any],
) -> None:
    """
    Set instructions for the genie space

    FAIR WARNING ALERT: the route and parameters are not public and may change at any time.
    I just used these because I didn't want to manually create a genie room
    """
    route_instruct = route + genie_space_id + "/instructions"
    print(route_instruct)
    resp = w.api_client.do(method="POST", path=route_instruct, body=payload_text_instructions)
    time.sleep(1)
    print(resp)
    resp = w.api_client.do(method="POST", path=route_instruct, body=pyaload_sql_instructions)
    print(resp)
    return

def get_or_create_genie_room(data_rooms: list[dict], payload: dict) -> str:
    """
    Retreive a genie room or create it if it doesn't exist.

    FAIR WARNING ALERT: the route and parameters are not public and may change at any time.
    I just used these because I didn't want to manually create a genie room
    """
    genie_space_name = payload["display_name"]
    if genie_space_name not in [i["display_name"] for i in data_rooms]:
        print("Creating genie space")
        resp = w.api_client.do(method="POST", path=route, body=payload)
        if genie_space_name == "retail-sales-store-performance":
            _post_sales_forecast_instructions(
                resp["id"], payload_sales_text_instruction, payload_sales_sql_query
            )
        return resp["id"]
    else:
        return next(
            i["id"] for i in data_rooms if i["display_name"] == genie_space_name
        )


product_genie_room_id = get_or_create_genie_room(
    data_rooms, payload_products_inventory_genie_room
)
sales_store_genie_room_id = get_or_create_genie_room(
    data_rooms, payload_sales_store_genie_room
)

print(f"Product Inventory Genie Room ID: {product_genie_room_id}")
print(f"Sales Store Genie Room ID: {sales_store_genie_room_id}")

Product Inventory Genie Room ID: 01f0408efb0c19099ce7e958563c175a
Sales Store Genie Room ID: 01f0408ecdfe1d2595651a15a88f909a



### Create Vector Store




#### Provision endpoint

In [0]:
import time

from databricks.vector_search.client import VectorSearchClient

client_v = VectorSearchClient()

try:
    resp = client_v.get_endpoint(vector_search_endpoint_name)
except Exception:
    print(f"Endpoint {vector_search_endpoint_name} not found. Creating...")
    client_v.create_endpoint(
        name=vector_search_endpoint_name,
        endpoint_type="STANDARD",  # or "STORAGE_OPTIMIZED"
    )

while True:
    resp = client_v.get_endpoint(vector_search_endpoint_name)
    if resp["endpoint_status"].get("state") == "PROVISIONING":
        print("Endpoint is provisioning...")
        time.sleep(30)
        continue
    print(f"Endpoint is {resp['endpoint_status']['state']}")
    break

print(resp)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.
Endpoint is ONLINE
{'name': 'vector-search-multi-agent-genie', 'creator': 'marshall.krassenstein@databricks.com', 'creation_timestamp': 1749046107451, 'last_updated_timestamp': 1749046107451, 'endpoint_type': 'STANDARD', 'last_updated_user': 'marshall.krassenstein@databricks.com', 'id': '633ac9c4-0ffd-4f95-a544-c8d7e94adf02', 'endpoint_status': {'state': 'ONLINE'}, 'num_indexes': 1}



#### Make vector search index

In [0]:
index_name = f"{catalog}.{schema}.retail_code_of_conduct_index"
try:
    index = client_v.get_index(vector_search_endpoint_name, index_name)

except Exception:
    spark.sql(f"""
    ALTER TABLE {catalog}.{schema}.retail_code_of_conduct
    SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
  """)

    index = client_v.create_delta_sync_index(
        endpoint_name=vector_search_endpoint_name,
        source_table_name=f"{catalog}.{schema}.retail_code_of_conduct",
        index_name=index_name,
        pipeline_type="TRIGGERED",
        primary_key="sec_id",
        embedding_source_column="text_chunks",
        embedding_model_endpoint_name="databricks-gte-large-en",
    )

while True:
    index_status = index.describe()
    if not index_status["status"]["ready"]:
        print(index_status["status"])
        time.sleep(30)
        continue
    break
index_status

{'name': 'mk_fiddles.genie_multi_agent.retail_code_of_conduct_index',
 'endpoint_name': 'vector-search-multi-agent-genie',
 'primary_key': 'sec_id',
 'index_type': 'DELTA_SYNC',
 'delta_sync_index_spec': {'source_table': 'mk_fiddles.genie_multi_agent.retail_code_of_conduct',
  'embedding_source_columns': [{'name': 'text_chunks',
    'embedding_model_endpoint_name': 'databricks-gte-large-en'}],
  'pipeline_type': 'TRIGGERED',
  'pipeline_id': 'fbcdefd6-81b2-4a06-9497-222f85891a65'},
 'status': {'detailed_state': 'ONLINE_NO_PENDING_UPDATE',
  'message': 'Index creation succeeded. Check latest status: https://adb-1720970340056130.10.azuredatabricks.net/explore/data/mk_fiddles/genie_multi_agent/retail_code_of_conduct_index',
  'indexed_row_count': 4,
  'triggered_update_status': {'last_processed_commit_version': 1,
   'last_processed_commit_timestamp': '2025-06-04T17:56:52Z'},
  'ready': True,
  'index_url': 'adb-1720970340056130.10.azuredatabricks.net/api/2.0/vector-search/indexes/mk_fidd

In [0]:
index = client_v.get_index(vector_search_endpoint_name, index_name)

index.similarity_search(
    query_text="What is our policy about children?",
    columns=["sec_id", "text_chunks"],
    num_results=5,
    filters={"sec_id NOT": ["1", "2", "5"]},
)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


{'manifest': {'column_count': 3,
  'columns': [{'name': 'sec_id'}, {'name': 'text_chunks'}, {'name': 'score'}]},
 'result': {'row_count': 2,
  'data_array': [[3.0,
    'Vendors/Suppliers must treat all employees with respect and dignity and provide them with a safe and healthy working environment.',
    0.0016412847],
   [4.0,
    "Factories shall comply with all applicable laws and regulations regarding the working environment, and must be free of any hazardous conditions. Vendors/Suppliers who provide residential accommodations for employees must apply similar standards to their residential facilities. Worker's Insurance\nWholesale Club prefers Vendors/Suppliers who provide appropriate health and life insurance for their employees. Vendors/Suppliers must comply with all appropriate local laws and regulations requiring health insurance, life insurance and worker's compensation. Environment\nWholesale Club is committed to preserving the environment. Vendors/Suppliers must comply with a


### MLFlow experiment


In [0]:
from pathlib import Path

import mlflow

mlflow_experiment_name = "agent_multi_tool_monitor"

mlflow_experiment = str(Path(".").absolute().parent / mlflow_experiment_name)

experiment = mlflow.get_experiment_by_name(mlflow_experiment.replace("/Workspace", ""))
if experiment is None:
    print("creating experiment..")
    experiment_id = mlflow.create_experiment(name=mlflow_experiment)
    experiment = mlflow.get_experiment(experiment_id)

experiment.experiment_id

'2933242203415888'

### Write configs to app_config.yaml

In [ ]:
import yaml

with open("app_config.yaml", "w") as f:
    yaml.dump(
        {
            "CATALOG": catalog,
            "SCHEMA": schema,
            "DATABRICKS_MODEL": databricks_foundation_model,
            "GENIE_SPACE_STORE_PERFORMANCE_ID": sales_store_genie_room_id,
            "GENIE_SPACE_PRODUCT_INV_ID": product_genie_room_id,
            "MLFLOW_EXPERIMENT_ID": experiment.experiment_id,
        },
        f,
        default_flow_style=False,
    )

### Download app_config.yaml

If this ran successfully, you should have all of the variables you need to run the streamlit app. 

To run locally, download the app_config.yaml file and run the streamlit app from your terminal with the following command:

```bash
streamlit run streamlit_multi_genie_tools.py
```

To deploy the application from your local terminal, run the following command:

```bash
databricks apps deploy $APP_NAME --source-code-path $APP_WORKSPACE_PATH
```